# Обеспечение качества кода: pre-commit, GitHub Actions, CI как quality gate 

## Quality Gates: Как не дать плохому коду убить ваш проект
    
В реальном мире качество кода — не «опция». Это **инфраструктурная необходимость**. Сегодня мы рассмотрим три уровня автоматической защиты:

1. **Pre-commit** 
    
2. **CI (GitHub Actions)** 

3. **Branch Protection Rules**
    
Вместе они образуют **Quality Gate** — систему, которая физически не позволяет плохому коду попасть в `main`.
   

## 1. Pre-commit
    
Pre-commit — это фреймворк для запуска проверок **перед каждым коммитом**. Он работает локально, но использует те же правила, что и CI.

### Установка:
```bash
    pip install pre-commit
    pre-commit install 
```

Теперь любой `git commit` запустит проверки.
  

In [ ]:
# Пример: .pre-commit-config.yaml
%%writefile .pre-commit-config.yaml

minimum_pre_commit_version: 3.0.0

repos:
  # ───────────────────────────────────────
  # 1. Стандартные хуки от pre-commit
  # ───────────────────────────────────────
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.5.0
    hooks:
      - id: trailing-whitespace        # Удаляет пробелы в конце строк
      - id: end-of-file-fixer          # Гарантирует пустую строку в конце файла
      - id: check-yaml                 # Проверяет синтаксис .yaml
      - id: check-json                 # Проверяет синтаксис .json
      - id: debug-statements           # Ловит случайные print в продакшене

  # ───────────────────────────────────────
  # 2. Ruff — всё в одном: линтер + форматтер
  # ───────────────────────────────────────
  - repo: https://github.com/astral-sh/ruff-pre-commit
    rev: v0.4.10  # ← ВСЕГДА проверяйте последнюю версию на GitHub!
    hooks:
      # Линтинг 
      - id: ruff
        args: [--fix, --exit-non-zero-on-fix]  # Исправляем автоматически, но требуем чистоты
      # Форматирование 
      - id: ruff-format

  # ───────────────────────────────────────
  # 3. Статическая типизация — ваша страховка от багов
  # ───────────────────────────────────────
  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.8.0  # ← Соответствует версии mypy в requirements-dev.txt
    hooks:
      - id: mypy
        # Указываем корневую директорию кода (замените на свою)
        args: [--strict]


Обратите внимание — мы не просто «запускаем линтеры». Мы **гарантируем единообразие**: форматирование, стиль, типы. Это экономит часы ревью.

##  2. GitHub Actions: CI как Quality Gate

Pre-commit — это хорошо, но недостаточно. Поэтому мы используем **CI** — Continuous Integration.
    
В GitHub Actions мы описываем pipeline, который запускается при каждом Pull Request."
   

In [ ]:
# Пример: .github/workflows/ci.yml
%%writefile .github/workflows/ci.yml
name: CI
    on:
      pull_request:
        branches: [ main ]

    jobs:
      quality:
        runs-on: ubuntu-latest
        steps:
          - uses: actions/checkout@v4
    
          - name: Set up Python
            uses: actions/setup-python@v5
            with:
              python-version: "3.11"

          - name: Install dependencies
            run: |
              python -m pip install --upgrade pip
              pip install ruff mypy pytest

          - name: Lint with Ruff
            run: ruff check .

          - name: Check formatting with Ruff
            run: ruff format --check .

          - name: Type check with mypy
            run: mypy .

          - name: Run tests with pytest
            run: pytest 

Этот pipeline — **ваш контракт с командой**. Если он зелёный — код можно ревьюить. Если красный — даже не тратьте время на ревью.

##  3. Branch Protection Rules 

Даже самый строгий CI бесполезен, если можно просто запушить в `main` напрямую.

Поэтому в GitHub:

1. Перейдите в **Settings → Branches**
2. Создайте правило для `main`
3. Включите:
    -  Require pull request reviews before merging
    -  Require status checks to pass before merging
    -  Require branches to be up to date before merging
    -  Include administrators



##  Почему это работает?

Потому что мы **автоматизируем доверие**:

- Pre-commit → доверие к себе
- CI → доверие к команде
- Branch protection → доверие к процессу

    Это не «накладные расходы». Это **ускорение разработки** за счёт предотвращения багов до их появления.


## Полезные ссылки

### CI:

<https://habr.com/ru/articles/508216/>

<https://habr.com/ru/articles/498436/>

<https://docs.gitlab.com/ci/>

<https://www.atlassian.com/continuous-delivery/continuous-integration>

### Pre-commit

<https://habr.com/ru/articles/498436/>

<https://blog.deepschool.ru/data/pre-commit-hook/>

<https://habr.com/ru/companies/nixys/articles/682612/>

<https://stefaniemolin.com/articles/devx/pre-commit/setup-guide/>